# Preparation and Reading Input Data

In [81]:
import sys
import numpy as np

def get_distance(Atom1, Atom2):
    # Calculate Euclidean distance. 1-D and 3-D in the future
    return np.sqrt(sum((Atom1 - Atom2) ** 2))

def n_dim_input_to_numpy_array(temp):
    temp = temp.split(',')
    temp = map(float,temp)
    return np.asfarray(temp)
    calc_count = calc_count +1

In [82]:
%timeit
#----------------------Reading Input File-------------------------------#
read_file = open('input.txt')
atoms = list()
for count, line in enumerate(read_file):
    if count == total_file_lines+1 or count >= reading_start_point_i+WINDOW_SIZE:
        break
    if count >= reading_start_point_i and count <reading_start_point_i+WINDOW_SIZE:
        atoms.append(n_dim_input_to_numpy_array(line))

In [83]:
WINDOW_SIZE = 145746
reading_start_point_i = 0
j_dim = 0
total_file_lines =  145746
cutoff = 15

In [84]:
# That means that we are not calculating the elements of the main diagonal which are the same.
# We do calculate differnt atoms
if reading_start_point_i!=j_dim:
    atomsY = list()
    atomsY.append(n_dim_input_to_numpy_array(line)) # already read from previous for-loop
    for countY, line in enumerate(read_file):
        if countY > j_dim + WINDOW_SIZE-count-1:
            break
        if countY >= j_dim-count and countY < j_dim + WINDOW_SIZE-count-1:  #-1 because we already appended the first line 
            atomsY.append(n_dim_input_to_numpy_array(line))
read_file.close()

In [86]:
atoms[:5]

[array([ 458.09997559,  510.39996338,   59.09999847]),
 array([ 453.69998169,  525.39996338,   53.5       ]),
 array([ 448.5       ,  524.39996338,   49.5       ]),
 array([ 434.29998779,  508.99996948,   56.59999847]),
 array([ 459.29998779,  516.79998779,   55.79999924])]

# Distance Calculation

In [87]:
a=atoms[:10]

## Giannis

In [88]:
%time
# the difference is that in the Cus compute data that are in main diagonal compute half of the elements 
# because table is symmetric, so the second loop can be half in the first case 

WINDOW_SIZE=len(a)
distances=np.empty((WINDOW_SIZE, WINDOW_SIZE), dtype='bool')

for i in range(0, WINDOW_SIZE):
    for j in range(i+1,WINDOW_SIZE):
        dist = get_distance(a[i],a[j])  
        if dist<=cutoff:
            distances[i][j]=True 
        else:
            distances[i][j]=False

np.save("distances_%d_%d.npz.npy" % (reading_start_point_i,j_dim),distances)

CPU times: user 2 µs, sys: 3 µs, total: 5 µs
Wall time: 16.9 µs


## Python Plain NumPy

In [89]:
import sklearn.metrics.pairwise
import numpy as np
import random
import sklearn.metrics.pairwise
import scipy.spatial.distance
from scipy.spatial.distance import cdist

In [90]:
%%time
a_np=np.stack((a), axis=0)

CPU times: user 164 µs, sys: 627 µs, total: 791 µs
Wall time: 2.85 ms


In [91]:
a_np

array([[ 458.09997559,  510.39996338,   59.09999847],
       [ 453.69998169,  525.39996338,   53.5       ],
       [ 448.5       ,  524.39996338,   49.5       ],
       [ 434.29998779,  508.99996948,   56.59999847],
       [ 459.29998779,  516.79998779,   55.79999924],
       [ 464.3999939 ,  504.59997559,   65.59999847],
       [ 449.59997559,  510.70001221,   59.09999847],
       [ 447.5       ,  501.69998169,   56.79999924],
       [ 447.89996338,  518.59997559,   52.79999542],
       [ 445.09997559,  541.20001221,   47.99999619]])

In [78]:
dist_sq = np.sqrt(np.sum((a_np[:,np.newaxis,:] - a_np[np.newaxis,:,:]) ** 2, axis=-1))

## Scikit

In [77]:
cdist(a_np, a_np, 'euclidean')

array([[   0.        ,   16.60481645,   19.50178202, ...,  655.08041998,
         651.44640112,  658.14603218],
       [  16.60481645,    0.        ,    6.63624966, ...,  669.47429722,
         665.89654672,  672.40594688],
       [  19.50178202,    6.63624966,    0.        , ...,  671.35132061,
         667.80512978,  674.14992307],
       ..., 
       [ 655.08041998,  669.47429722,  671.35132061, ...,    0.        ,
           6.51844584,   17.1149026 ],
       [ 651.44640112,  665.89654672,  667.80512978, ...,    6.51844584,
           0.        ,   23.00107492],
       [ 658.14603218,  672.40594688,  674.14992307, ...,   17.1149026 ,
          23.00107492,    0.        ]])

# MXNet

In [74]:
import mxnet as mx

In [92]:
a_mx=mx.nd.array(a_np)
diff=(mx.ndarray.power(a_mx.reshape((1, a_mx.shape[0], a_mx.shape[1])) - a_mx.reshape((a_mx.shape[0], 1, a_mx.shape[1])), 2))
dist=mx.ndarray.sqrt(mx.ndarray.sum_axis(diff, axis=2)).asnumpy()

In [93]:
dist

array([[  0.        ,  16.60481644,  19.50178337,  23.97184563,
          7.30002308,  10.75081825,   8.50529385,  13.90464497,
         14.52482319,  35.22574615],
       [ 16.60481644,   0.        ,   6.63624954,  25.59159088,
         10.51711178,  26.33514214,  16.25603294,  24.71879768,
          8.96493435,  18.81094551],
       [ 19.50178337,   6.63624954,   0.        ,  22.1180954 ,
         14.63179302,  30.06757736,  16.76480293,  23.86585808,
          6.69999027,  17.20615578],
       [ 23.97184563,  25.59159088,  22.1180954 ,   0.        ,
         26.20076942,  31.72333908,  15.59582615,  15.08542824,
         17.0751133 ,  35.03487778],
       [  7.30002308,  10.51711178,  14.63179302,  26.20076942,
          0.        ,  16.45874786,  11.92434216,  19.18983841,
         11.92478657,  29.2889328 ],
       [ 10.75081825,  26.33514214,  30.06757736,  31.72333908,
         16.45874786,   0.        ,  17.27718163,  19.27329254,
         25.14142036,  44.96458054],
       [  

# Tensorflow

In [94]:
import tensorflow as tf

In [95]:
tf.__version__

'0.11.0rc0'

# Testing and Playing around with Numpy Broadcasting

In [10]:
a_np = np.arange(6).reshape(2,3)
a_np

array([[0, 1, 2],
       [3, 4, 5]])

In [67]:
x=a_np[:, np.newaxis, :]
y=a_np[np.newaxis,:,:]

In [69]:
y.shape

(1, 2, 3)

In [70]:
(x-y).shape

(2, 2, 3)